## Table of Contents

1) Import Libraries and Data

2) Find Mean order number by department

3) Create loyalty flag

4) Find descriptive stats by loyalty flag

5) Create spending flag

6) Create order frequency flag

7) Export dataframe

### 1)
Import Libraries and Data

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#define path to import data from
path = r'C:\Users\Owner\Documents\Career Foundry\Tasks\Data Immersion Tasks\Instacart Project\2 Data'

In [3]:
#import merged orders and products data with additional counts as ords_prods_merge
ords_prods_merge = pd.read_pickle(os.path.join(path, 'prepared data', 'order product merge with price ranges and busy counts.pkl'))

### 2)
In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [4]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range,busiest_day,busiest_2_days,busiest_period_of_day
0,2539329.0,1.0,1.0,2.0,8.0,NaN,196,1.0,0.0,Soda,77.0,7.0,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2398795.0,1.0,2.0,3.0,7.0,15.0,196,1.0,1.0,Soda,77.0,7.0,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders
2,473747.0,1.0,3.0,3.0,12.0,21.0,196,1.0,1.0,Soda,77.0,7.0,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders
3,2254736.0,1.0,4.0,4.0,7.0,29.0,196,1.0,1.0,Soda,77.0,7.0,9.0,both,Mid-range product,Least busy,Least busy,Average orders
4,431534.0,1.0,5.0,4.0,15.0,28.0,196,1.0,1.0,Soda,77.0,7.0,9.0,both,Mid-range product,Least busy,Least busy,Most orders


In [5]:
#find mean for order_number grouped by department
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1.0,15.457838
2.0,17.277920
3.0,17.170395
4.0,17.811403
5.0,15.215751
6.0,16.439806
7.0,17.225802
8.0,15.340650


### Analyze the result. 
How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

To start with there are a lot more departments in the full dataframe. The subset only had 8 departments while the full dataframe has 21.
The means for the departments in both have also changed. This is usually up or down by about 0-2. Departments 16 and 17 look to be the biggest changers going from 19.5 to 17.7 for department 16 and 11.3 to 15.7 for department 17.

### 3)
Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [6]:
#create a column 'max_order' in ords_prods_merge and put the highest amount of orders by each user id
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)

In [7]:
#use following 3 lines to assign loyalty levels in new column loyalty_flag to user ids based on max_order
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [8]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] > 10) & (ords_prods_merge['max_order'] <= 40), 'loyalty_flag'] = 'Regular customer'

In [9]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New Customer'

In [25]:
#check counts in 'loyalty_flag' column in ords_prods_merge
ords_prods_merge['loyalty_flag'].value_counts(dropna=False)

Regular customer    15891507
Loyal customer      10294027
New Customer         6249525
NaN                       11
Name: loyalty_flag, dtype: int64

### 4)
The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [10]:
#group by customer loyalty and find basic statistics of prices for each group
ords_prods_merge.groupby('loyalty_flag').agg({'prices':['min','max','mean']})

prices                    
                    min      max       mean
loyalty_flag                               
Loyal customer      1.0  99999.0  10.386336
New Customer        1.0  99999.0  13.294670
Regular customer    1.0  99999.0  12.495717

### 5) 
The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [11]:
#create a column avg_user_price and populate it with the average price of all products purchased by each user
ords_prods_merge['avg_user_price'] = ords_prods_merge.groupby('user_id')['prices'].transform(np.mean)

In [12]:
#create test subset of data for user_id 19 
df_user19 = ords_prods_merge[ords_prods_merge['user_id']==19]

In [13]:
#check mean of prices for user df_user19
df_user19.agg({'prices':['mean']})

,prices
mean,8.372059


In [14]:
#confirm against ords_prods_merge that mean shows as 8.4
ords_prods_merge.head(20)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,department_id,prices,_merge,price_range,busiest_day,busiest_2_days,busiest_period_of_day,max_order,loyalty_flag,avg_user_price
0,2539329.0,1.0,1.0,2.0,8.0,NaN,196,1.0,0.0,Soda,...,7.0,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10.0,New Customer,6.367797
1,2398795.0,1.0,2.0,3.0,7.0,15.0,196,1.0,1.0,Soda,...,7.0,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10.0,New Customer,6.367797
2,473747.0,1.0,3.0,3.0,12.0,21.0,196,1.0,1.0,Soda,...,7.0,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10.0,New Customer,6.367797
3,2254736.0,1.0,4.0,4.0,7.0,29.0,196,1.0,1.0,Soda,...,7.0,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10.0,New Customer,6.367797
4,431534.0,1.0,5.0,4.0,15.0,28.0,196,1.0,1.0,Soda,...,7.0,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10.0,New Customer,6.367797
5,3367565.0,1.0,6.0,2.0,7.0,19.0,196,1.0,1.0,Soda,...,7.0,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10.0,New Customer,6.367797
6,550135.0,1.0,7.0,1.0,9.0,20.0,196,1.0,1.0,Soda,...,7.0,9.0,both,Mid-range product,Regularly busy,Busiest day,Most orders,10.0,New Customer,6.367797
7,3108588.0,1.0,8.0,1.0,14.0,14.0,196,2.0,1.0,Soda,...,7.0,9.0,both,Mid-range product,Regularly busy,Busiest day,Most orders,10.0,New Customer,6.367797
8,2295261.0,1.0,9.0,1.0,16.0,0.0,196,4.0,1.0,Soda,...,7.0,9.0,both,Mid-range product,Regularly busy,Busiest day,Most orders,10.0,New Customer,6.367797
9,2550362.0,1.0,10.0,4.0,8.0,30.0,196,1.0,1.0,Soda,...,7.0,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10.0,New Customer,6.367797


In [15]:
#use following 2 lines to assign spend levels in new column spend_flag to user ids based on avg_user_price
ords_prods_merge.loc[ords_prods_merge['avg_user_price'] >= 10, 'spend_flag'] = 'High Spender'

In [16]:
ords_prods_merge.loc[ords_prods_merge['avg_user_price'] < 10, 'spend_flag'] = 'Low Spender'

### 6)
In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [17]:
#create a column median_days_since_order and populate it with the median of 'days_since_prior_order' of all orders for each user
ords_prods_merge['median_days_since_order'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)

In [18]:
#create test subset of data for user_id 18 
df_user18 = ords_prods_merge[ords_prods_merge['user_id']==18]

In [19]:
#check median of median_days_since_order for user df_user18
df_user18['median_days_since_order'].median()

8.0

In [20]:
#visually confirm by looking at subset that median can be 8
df_user18

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,price_range,busiest_day,busiest_2_days,busiest_period_of_day,max_order,loyalty_flag,avg_user_price,spend_flag,median_days_since_order
3411902,1020460.0,18.0,5.0,2.0,16.0,9.0,21137,13.0,0.0,Organic Strawberries,...,both,Mid-range product,Regularly busy,Regularly busy,Most orders,6.0,New Customer,7.702564,Low Spender,8.0
4303617,3106101.0,18.0,2.0,0.0,17.0,1.0,31717,5.0,0.0,Organic Cilantro,...,both,Low-range product,Busiest day,Busiest day,Average orders,6.0,New Customer,7.702564,Low Spender,8.0
5760167,1020460.0,18.0,5.0,2.0,16.0,9.0,26209,3.0,0.0,Limes,...,both,Mid-range product,Regularly busy,Regularly busy,Most orders,6.0,New Customer,7.702564,Low Spender,8.0
7735129,1020460.0,18.0,5.0,2.0,16.0,9.0,34126,2.0,0.0,Organic Italian Parsley Bunch,...,both,Mid-range product,Regularly busy,Regularly busy,Most orders,6.0,New Customer,7.702564,Low Spender,8.0
8553439,3106101.0,18.0,2.0,0.0,17.0,1.0,5876,3.0,0.0,Organic Lemon,...,both,Low-range product,Busiest day,Busiest day,Average orders,6.0,New Customer,7.702564,Low Spender,8.0
10094346,2780889.0,18.0,1.0,6.0,18.0,NaN,25997,1.0,0.0,Doctor Formulated Probiotics Once Daily Women'...,...,both,Low-range product,Regularly busy,Regularly busy,Average orders,6.0,New Customer,7.702564,Low Spender,8.0
10094498,2780889.0,18.0,1.0,6.0,18.0,NaN,47546,2.0,0.0,Chocolate Coconut Milk Beverage,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,6.0,New Customer,7.702564,Low Spender,8.0
10094499,1860960.0,18.0,3.0,1.0,19.0,8.0,47546,2.0,1.0,Chocolate Coconut Milk Beverage,...,both,Mid-range product,Regularly busy,Busiest day,Average orders,6.0,New Customer,7.702564,Low Spender,8.0
10094500,3133044.0,18.0,4.0,0.0,16.0,6.0,47546,1.0,1.0,Chocolate Coconut Milk Beverage,...,both,Mid-range product,Busiest day,Busiest day,Most orders,6.0,New Customer,7.702564,Low Spender,8.0
10094501,1020460.0,18.0,5.0,2.0,16.0,9.0,47546,1.0,1.0,Chocolate Coconut Milk Beverage,...,both,Mid-range product,Regularly busy,Regularly busy,Most orders,6.0,New Customer,7.702564,Low Spender,8.0


In [21]:
#use following 3 lines to assign frequency flags in new column frequency_flag to user ids based on median_days_since_order
ords_prods_merge.loc[ords_prods_merge['median_days_since_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [22]:
ords_prods_merge.loc[(ords_prods_merge['median_days_since_order'] > 10) & (ords_prods_merge['median_days_since_order'] <= 20), 'frequency_flag'] = 'Regular customer'

In [23]:
ords_prods_merge.loc[ords_prods_merge['median_days_since_order'] <= 10, 'frequency_flag'] = 'Frequent customer'

### 7)
Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [24]:
#export data frame as 'order product merge with price ranges-busy counts-user flags.pkl'
ords_prods_merge.to_pickle(os.path.join(path, 'prepared data', 'order product merge with price ranges-busy counts-user flags.pkl'))